In [1]:
import pandas as pd
import os, glob, re, shutil
import PIL.ExifTags as ExifTags
import PIL.Image as Image
import numpy as np
from tqdm import tqdm
from typing import Union
import piexif

# Naming conventions

In [4]:
def get_exif(image_path: Union[str, ]) -> dict:
    with Image.open(image_path) as image:
        exif_data = image._getexif()
        if not exif_data:
            return {}

        exif = {}
        for tag, value in exif_data.items():
            decoded = ExifTags.TAGS.get(tag, tag)
            exif[decoded] = value
        return exif

def get_exif_gps(exif_info: dict):
    gps_info = {}

    for key in exif_info['GPSInfo'].keys():
        decode = ExifTags.GPSTAGS.get(key,key)
        gps_info[decode] = exif_info['GPSInfo'][key]#int.from_bytes(exif_info['GPSInfo'][key], byteorder='big')
    return gps_info

def get_exif_orientation(img_path: str):
    """
    In EXIF at id 274 hexadecimal 0x0112, there is a tag called Orientation
    1 = Horizontal (normal)
    2 = Mirror horizontal
    3 = Rotate 180
    4 = Mirror vertical
    5 = Mirror horizontal and rotate 270 CW
    6 = Rotate 90 CW
    7 = Mirror horizontal and rotate 90 CW
    8 = Rotate 270 CW
    

    In PIL
    FLIP_LEFT_RIGHT = 0
    FLIP_TOP_BOTTOM = 1
    ROTATE_180 = 3
    ROTATE_270 = 4
    ROTATE_90 = 2
    TRANSPOSE = 5
    TRANSVERSE = 6
    """
    # iamge.transpose comaptible dictionary
    # https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.transpose
    transpose_map = {
        1: None,                    # Horizontal (normal)
        2: Image.FLIP_LEFT_RIGHT,   # Mirror horizontal
        3: Image.ROTATE_180,        # Rotate 180
        4: Image.FLIP_TOP_BOTTOM,   # Mirror vertical
        5: Image.TRANSPOSE,         # Mirror horizontal and rotate 270 CW
        6: Image.ROTATE_270,        # Rotate 90 CW value of 4
        7: Image.TRANSVERSE,        # Mirror horizontal and rotate 90 CW
        8: Image.ROTATE_90          # Rotate 270 CW value of 2
    }
    
    with Image.open(img_path) as img:
        try:
            orientation = img._getexif().get(274, 0)
            # print(f"img_path: {img_path}, orientation: {orientation}")
        except AttributeError:
            return None
        
    return transpose_map[orientation]


In [107]:
exif_info = get_exif("/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/only_6/marie_PL-AH4_bagonly-2_19.JPG")
# exif_info = get_exif(all_jpgs[0])
if exif_info == {}:
    print("No EXIF data found for image")
    exit()
    
for tag, value in exif_info.items():

    if tag == 'GPSInfo':
        print("GPSInfo")
        gps_info = get_exif_gps(exif_info)
        for tag, value in gps_info.items():
            print(f"{' ' * 5}{tag}: {value}")

    else:
        print(f"{tag}: {value}")
    

GPSInfo
     GPSVersionID: b'\x02\x03\x00\x00'
ResolutionUnit: 2
ExifOffset: 213
Make: Canon
Model: Canon EOS RP
YResolution: 72.0
Orientation: 1
DateTime: 2024:07:04 12:57:07
YCbCrPositioning: 2
Copyright: 
XResolution: 72.0
Artist: 
ExifVersion: b'0231'
ComponentsConfiguration: b'\x01\x02\x03\x00'
ShutterSpeedValue: 0.0
DateTimeOriginal: 2024:07:04 12:57:07
DateTimeDigitized: 2024:07:04 12:57:07
ApertureValue: 7.375
ExposureBiasValue: 0.0
MeteringMode: 2
UserComment: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [112]:
all_jpgs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/only_6/*.JPG')
print(f"total images: {len(all_jpgs)}")


parent_dir= os.path.dirname(all_jpgs[0])
print(f"parent directory: {parent_dir}")


total images: 50
parent directory: /cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/only_6


In [116]:

# make a dataframe with 3 columns
camera_names_df= pd.DataFrame(columns=['tower', 'aim', 'name', 'focal_length', 'Width', 'Height', 'Camera_model', 'Orientation'])


for i, jpg in enumerate(all_jpgs):
    camera_name= re.split('[-_]', os.path.basename(jpg))[2]
    exif_info = get_exif(jpg)
    
    # Create a dictionary with the new data
    try:
        camera_names_df.loc[len(camera_names_df)] = {
            'tower': camera_name[0],
            'aim': camera_name[1],
            'name': camera_name[2],
            'focal_length': exif_info['FocalLength'],
            'Width': exif_info['ExifImageWidth'],
            'Height': exif_info['ExifImageHeight'],
            'Camera_model': exif_info['LensModel'],
            'Orientation': exif_info['Orientation']
        }
    except KeyError:
        print(f"KeyError: {i}, {jpg}")
        
    # if camera_name == 'KF1':
    #     print(i, exif_info['LensModel'])
    
# :NOTE change i to I
camera_names_df['tower']= camera_names_df['tower'].str.upper()


In [117]:
camera_names_df

,tower,aim,name,focal_length,Width,Height,Camera_model,Orientation
0,A,H,4,131.0,6240,4160,EF70-200mm f/4L USM,1
1,A,H,6,126.0,6240,4160,EF70-200mm f/4L USM,1
2,A,H,3,131.0,6240,4160,EF70-200mm f/4L USM,1
3,A,H,2,131.0,6240,4160,EF70-200mm f/4L USM,1
4,A,T,1,98.0,6240,4160,EF70-200mm f/4L USM,1
5,A,T,2,94.0,6240,4160,EF70-200mm f/4L USM,1
6,B,H,1,131.0,6240,4160,EF70-200mm f/4L USM,1
7,B,H,3,200.0,6240,4160,EF70-200mm f/4L USM,1
8,B,H,4,200.0,6240,4160,EF70-200mm f/4L USM,1
9,B,H,5,98.0,6240,4160,EF70-200mm f/4L USM,1


In [14]:
# camera_names_df.to_csv('camera_names.csv', index=False)

In [ ]:
# group by width, height, focal_length, tower, aim

group_df= pd.pivot_table(camera_names_df, index=['Width', 'Height', 'focal_length', 'tower', 'aim'], aggfunc='count')
display(group_df)

# Rotate each image and change exif tag

In [58]:
import piexif


In [115]:
transpose_map = {
        1: None,                    # Horizontal (normal)
        2: Image.FLIP_LEFT_RIGHT,   # Mirror horizontal
        3: Image.ROTATE_180,        # Rotate 180
        4: Image.FLIP_TOP_BOTTOM,   # Mirror vertical
        5: Image.TRANSPOSE,         # Mirror horizontal and rotate 270 CW
        6: Image.ROTATE_270,        # Rotate 90 CW value of 4
        7: Image.TRANSVERSE,        # Mirror horizontal and rotate 90 CW
        8: Image.ROTATE_90          # Rotate 270 CW value of 2
    }


for jpg in tqdm(all_jpgs):
    with Image.open(jpg) as img:
        
        # exif_info = get_exif(jpg)
        # print(exif_info)
        exif_dict = piexif.load(img.info['exif'])
        # print(exif_dict)
        # orientation = get_exif_orientation(jpg)
        orientation= exif_dict.get('0th', {}).get(274, 1)
        # print(orientation, img._getexif().get(274, 0))

        if orientation != 1:
            orientation= transpose_map[orientation]
            exif_dict['0th'][274]= 1
        #     # print(exif_dict)

        #     with Image.open(jpg) as img:
            img= img.transpose(orientation)

                

            exif_bytes = piexif.dump(exif_dict)
            base, ext = os.path.splitext(jpg)
            new_fname = f"{base}{ext}"
            # print(new_fname)
            img.save(new_fname, "jpeg", exif=exif_bytes)

    #             # img.save(jpg)
    #             # print(f"orientation: {orientation}, img_path: {jpg}")
    #     # break


100%|██████████| 50/50 [00:07<00:00,  6.39it/s]


# make custom match text file

parameters to take
* focal length
* no. of towers on either side to consider

In [56]:
from typing import List
def n_closest_towers(tower, n:int= 1,)-> List[str]:
    """
    This function returns a list of n towers closest to the given tower in a clockwise direction.
    The tower names are represented by capital letters from 'A' to '
    """
    
    clock_tower_ring= {"A":"B", "B":"C", "C":"D", "D":"E", "E":"F", "F":"G", "G":"H", "H":"i", "i":"J", "J":"K","K":"A"}

    next_tower= tower
    tower_list= [next_tower]
    for i in range(n):
        next_tower= clock_tower_ring[next_tower]
        tower_list.append(next_tower)
    
    anticlockwise_tower_ring = {value: key for key, value in clock_tower_ring.items()}
    next_tower= tower
    for i in range(n):
        next_tower= anticlockwise_tower_ring[next_tower]
        tower_list.append(next_tower)
    
    return tower_list

# simple tests for n_closest_towers function 
print(n_closest_towers('A', 2))
print(n_closest_towers('B', 3))
print(n_closest_towers('H', 2))


['A', 'B', 'C', 'K', 'J']
['B', 'C', 'D', 'E', 'A', 'K', 'J']
['H', 'i', 'J', 'G', 'F']


In [36]:
a= re.split("[._-]",os.path.basename(all_jpgs[0]))
# b= a
b= a[-4]+"_"+a[-3]+"_"+a[-2]+"."+a[-1]
b

'AB1_dress3+shoe_15.JPG'

In [57]:
focal_length= str(float(35))
num_towers= 1
scene_name= "dress3+shoe_15"

all_jpgs= glob.glob(f"{parent_dir}/focal_length/{focal_length}/*.JPG")
print(f"total images: {len(all_jpgs)}")
print(all_jpgs[0])

total images: 4
C:\Users\student1\Desktop\Datasets\dress3_15/focal_length/35.0\marie_PL-AB6_dress3+shoe_15.JPG


In [58]:
all_jpgs= glob.glob(f"{parent_dir}/focal_length/{focal_length}/marie_*-i*_{scene_name}.JPG")
print(f"total images: {len(all_jpgs)}")
all_jpgs

total images: 0


[]

In [59]:
os.makedirs(f"{parent_dir}/COLMAP_models/focal_length/{focal_length}", exist_ok=True)
txt_path= f"{parent_dir}/COLMAP_models/focal_length/{focal_length}/tower_{num_towers}.txt"

all_jpgs= glob.glob(f"{parent_dir}/focal_length/{focal_length}/*.JPG")
# get all jpgs with the same focal length

with open(txt_path, 'w') as f:
    for i, jpg in enumerate(all_jpgs):
        camera_name= re.split('[-_]', os.path.basename(jpg))[2]
        tower= camera_name[0]
        n_closest_towers_list= n_closest_towers(tower, num_towers)
        for tower in n_closest_towers_list:
            # list of all jpgs with structure like this
            nearest_tower_jpgs= glob.glob(f"{parent_dir}/focal_length/{focal_length}/marie_*-{tower}*_{scene_name}.JPG")
            for near_tower_jpg in nearest_tower_jpgs:
                f.write(f"{os.path.basename(jpg)} {os.path.basename(near_tower_jpg)}\n")

# print done
print(f"Done writing to {txt_path}")

Done writing to C:\Users\student1\Desktop\Datasets\dress3_15/COLMAP_models/focal_length/35.0/tower_1.txt


In [7]:

# open png image
from PIL import Image


all_pngs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/segmented/*.png')
all_jpgs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/images/*.JPG')

print(f"total images: {len(all_pngs)}")
parent_dir= os.path.dirname(os.path.dirname(all_pngs[0]))
mask_path= os.path.join(parent_dir, 'mask_unrotated')
os.makedirs(mask_path, exist_ok=True)

for jpg, png in tqdm(zip(all_jpgs, all_pngs)):
    
    with Image.open(png) as img:
        img_name= os.path.basename(png).replace('png', 'JPG.png')
        np_img= np.asarray(img)

        alpha_channel= np_img[:,:,3]
        # Create a mask where transparent pixels are black (0) and non-transparent pixels are white (255)
        mask = np.where(alpha_channel > 0, 255, 0).astype(np.uint8)

        # Convert the mask to an image
        mask_img = Image.fromarray(mask, mode='L') # mode L is for 8-bit pixels, black and white
        rotation= get_exif_orientation(jpg)
        # print(type(rotation))
        # rotate the image based on the orientation
        # mask_img= mask_img.transpose(rotation)
        # print(f"{img.size} {mask_img.size}")

        mask_img.save(f"{mask_path}/{img_name}")
        # break

total images: 50


50it [00:25,  1.95it/s]


In [22]:
# 4th channel is alpha channel
# make a jpg mask with just the alpha channel


# save the alpha channel as a jpg
parent_dir= os.path.dirname(os.path.dirname(img_name))
os.makedirs(os.path.join(parent_dir, 'mask'), exist_ok=True)
mask_img_path= os.path.join(parent_dir, 'mask',os.path.basename(img_name).replace('png', 'jpg')) 

mask_img= Image.fromarray(alpha_channel)
mask_img.save(mask_img_path)

In [34]:

# iterate through each file and change .jpg. to .JPG.
all_jpgs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/handbag_19/visible_model/mask/*.jpg')
for jpg in tqdm(all_jpgs):
    new_jpg= jpg.replace('.jpg.', '.JPG.')
    os.rename(jpg, new_jpg)

100%|██████████| 50/50 [00:00<00:00, 337.81it/s]
